# script that links the loops with the enhancers and how active they are. 
basically same as loop correlation but saves the results in a different format so I can annotate genes

## to save time it will only do ATAC peaks near allelic imbalance and significant caQTL

In [1]:
import pandas as pd
import numpy as np
import hicstraw 
from multiprocessing import Pool
from functools import partial
import glob
import os
import plotly.express as px
import math
import matplotlib.pyplot as plt
from matplotlib import colors
from pandarallel import pandarallel
import cooler
import cooltools
import pybedtools as pbed
pandarallel.initialize()
from scipy import stats, special
from statsmodels.stats import multitest
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.io as pio
import seaborn as sns
os.makedirs("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/", exist_ok = True)
pbed.helpers.set_tempdir("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/")
bed_genome_file = "/mnt/iusers01/jw01/mdefscs4/hg38.genome"

plt.rcParams['svg.fonttype'] = 'none'

base_dir = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis"

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
gtf_annotation_df = pd.read_pickle(f"{base_dir}/gencode_gtf.pickle")
gtf_transcripts = gtf_annotation_df[(gtf_annotation_df["feature"] == "transcript") & (gtf_annotation_df["transcript_type"] == "protein_coding")].dropna(axis=1, how='all')
gtf_transcripts["gene_id"] = gtf_transcripts["gene_id"].str.split(".").str[0]
gtf_transcripts["transcript_id"] = gtf_transcripts["transcript_id"].str.split(".").str[0]
gtf_transcripts["TSS_start"] = gtf_transcripts.apply(lambda x: int(x["start"]) if x["strand"] == "+" else int(x["end"]) ,axis = 1)
gtf_transcripts = gtf_transcripts[gtf_transcripts["seqname"].str.startswith("chr")]

In [3]:
RNA_normalized_counts = pd.read_csv(f"{base_dir}/RNA_seq_analysis/RNA_normalized_counts.csv")
metadata_RNA = pd.read_csv(f"{base_dir}/metadata/cleaned_RNA_metadata.csv", index_col=0)
column_name_dict = dict(zip(metadata_RNA['sample'], metadata_RNA['proper_name']))
RNA_normalized_counts = RNA_normalized_counts.rename(columns=column_name_dict)
RNA_normalized_counts_melted = pd.melt(RNA_normalized_counts, id_vars=["ensembl","ENSG","symbol","genename","entrez"], 
        value_vars=RNA_normalized_counts.columns.difference(["ensembl","ENSG","symbol","genename","entrez"]),
        var_name="proper_name",value_name="expression")
RNA_normalized_counts_melted = RNA_normalized_counts_melted.merge(metadata_RNA[["patient","cell_type","condition","proper_name"]], on = "proper_name")

# RNA_normalized_counts_melted = RNA_normalized_counts_melted[RNA_normalized_counts_melted["cell_type"].isin(["CD4","CD8"])]

In [4]:
RNA_diff = pd.read_csv(f"{base_dir}/RNA_seq_analysis/DEseq2_result/DE_CD8_vs_CD4_all.csv", index_col = 0)

In [5]:
ATAC_normalised_counts = pd.read_csv(f"{base_dir}/ATAC_seq_analysis/ATAC_DESeq2_quantile_normalized_counts.csv", index_col = 0)
metadata_ATAC = pd.read_csv(f"{base_dir}/metadata/cleaned_ATAC_metadata.csv", index_col=0)
column_name_dict = dict(zip(metadata_ATAC['id'], metadata_ATAC['proper_name']))
ATAC_normalised_counts = ATAC_normalised_counts.rename(columns=column_name_dict)

ATAC_normalised_counts_melted = pd.melt(ATAC_normalised_counts, id_vars=["CHR","START","END"], 
        value_vars=ATAC_normalised_counts.columns.difference(["CHR","START","END"]),
        var_name="proper_name",value_name="peak_height")
ATAC_normalised_counts_melted = ATAC_normalised_counts_melted.merge(metadata_ATAC[["patient","cell_type","condition","proper_name"]], on="proper_name", suffixes = ["", "_y"])

# ATAC_normalised_counts_melted = ATAC_normalised_counts_melted[ATAC_normalised_counts_melted["cell_type"].isin(["CD4","CD8"])]

In [6]:
ATAC_diff = pd.read_csv(f"{base_dir}/ATAC_seq_analysis/diffbind_result/DE_CD8_vs_CD4_all.csv", index_col = 0)

In [7]:
loops_analysed = pd.read_pickle(f"{base_dir}/HiC_analysis/extracting_loop_counts/aggregated_counts/aggregated_normalized_loops_CD4_CD8.pk")
metadata_hic = pd.read_csv(f"{base_dir}/metadata/cleaned_HiC_metadata.csv", index_col = 0)
column_name_dict = dict(zip(metadata_hic['folder_name'], metadata_hic['proper_name']))
loops_analysed = loops_analysed.rename(columns=column_name_dict)
loops_analysed["chrA"] = "chr" + loops_analysed["chrA"]
loops_analysed["chrB"] = "chr" + loops_analysed["chrB"]
loops_analysed = loops_analysed.reset_index()
loops_counts_melted = pd.melt(loops_analysed, id_vars=['chrA', 'A_start', 'A_end', 'chrB', 'B_start', 'B_end', 'FDR', 'DETECTION_SCALE', 'distance_bin', 'index'], 
        value_vars=loops_analysed.columns.difference(['chrA', 'A_start', 'A_end', 'chrB', 'B_start', 'B_end', 'FDR', 'DETECTION_SCALE', 'distance_bin', 'index']),
        var_name="proper_name",value_name="interaction_strength")

loops_counts_melted = loops_counts_melted.merge(metadata_hic[["patient","cell_type","condition","proper_name"]], on = "proper_name")

## get the list of peaks that is a caQTL

In [8]:
ATAC_permuted_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/ATAC/output_final/ATAC_permuted_CD8_merged.txt", sep = " ")
ATAC_permuted_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/ATAC/output_final/ATAC_permuted_CD4_merged.txt", sep = " ")
ATAC_permuted_CD8 = ATAC_permuted_CD8[ATAC_permuted_CD8["adj_beta_pval"] < 0.1]
ATAC_permuted_CD4 = ATAC_permuted_CD4[ATAC_permuted_CD4["adj_beta_pval"] < 0.1]

In [9]:
ID_caQTL = set(ATAC_permuted_CD8["phe_id"].to_list() + ATAC_permuted_CD4["phe_id"].to_list())

## get the list of peaks that are very near allelic imbalance

In [10]:
all_SNPs_all = pd.read_pickle(f"{base_dir}/ATAC_allelic_imbalance/combined_p_vals_files/all_SNPs_all.pkl")
all_SNPs_all = all_SNPs_all[(all_SNPs_all["corrected_p_val_greater"] < 0.1) | (all_SNPs_all["corrected_p_val_less"] < 0.1)]

In [11]:
snps_anno_pbed = pbed.BedTool.from_dataframe(all_SNPs_all[["CHROM","POS","POS","ID"]])

In [12]:
ATAC_peaks = pbed.BedTool.from_dataframe(ATAC_normalised_counts.reset_index()[["CHR","START","END", "index"]])

res_anno_peaks = snps_anno_pbed.sort().closest(ATAC_peaks.sort(), d = True).to_dataframe(header = None, disable_auto_names = True)
res_anno_peaks = res_anno_peaks[res_anno_peaks[8] < 500]

In [13]:
ID_allelic_imb = set(res_anno_peaks[7].to_list())

get all the peaks worth analysing

In [14]:
ID_all = list(ID_caQTL.union(ID_allelic_imb))

In [15]:
ATAC_normalised_counts_filtered = ATAC_normalised_counts.loc[ID_all].copy()
ATAC_normalised_counts_filtered = ATAC_normalised_counts_filtered.reset_index()
ATAC_normalised_counts_melted = pd.melt(ATAC_normalised_counts_filtered, id_vars=["CHR","START","END", "index"], 
        value_vars=ATAC_normalised_counts_filtered.columns.difference(["CHR","START","END", "index"]),
        var_name="proper_name",value_name="peak_height")
ATAC_normalised_counts_melted = ATAC_normalised_counts_melted.merge(metadata_ATAC[["patient","cell_type","condition","proper_name"]], on="proper_name", suffixes = ["", "_y"])

## now we need to identify all the loops to test for each ATAC-peak

In [16]:
ATAC_peaks = pbed.BedTool.from_dataframe(ATAC_normalised_counts_filtered[["CHR","START","END", "index"]])
loops = pbed.BedTool.from_dataframe(loops_analysed[['chrA', 'A_start', 'A_end', 'chrB', 'B_start', 'B_end', "index"]])

In [17]:
loops_with_peaks = loops.pairtobed(ATAC_peaks.slop(b=5000, g=bed_genome_file), type = "either").to_dataframe(header = None, disable_auto_names = True)

In [18]:
loops_with_peaks.columns = ['chrA', 'A_start', 'A_end', 'chrB', 'B_start', 'B_end', "index_loop", "CHR","START","END", "index_peak"]

In [19]:
# correct the SLOP on the ATAC peaks
loops_with_peaks["START"] = loops_with_peaks["START"] + 5000
loops_with_peaks["END"] = loops_with_peaks["END"] - 5000

## collect the p-values and regression coefficients

In [20]:
def retrieve_p_vals(peak_values, loop_values):
    this_peak = peak_values.merge(loop_values[["proper_name","interaction_strength"]], on = "proper_name")
    s = smf.ols("interaction_strength~peak_height", data = this_peak) # interaction_strength~peak_height+cell_type+condition
    r = s.fit()
    return pd.Series({"p_value":r.pvalues.T["peak_height"],
                        "beta":r.params.T["peak_height"],
                        "R_squared":r.rsquared,
                        "R_squared_ajd":r.rsquared_adj,
                        "interaction_mean": loop_values["interaction_strength"].mean(),
                        "peak_mean":peak_values["peak_height"].mean(),})

In [21]:
def annotate_regression(row):
    index_peak = row["index_peak"]
    index_loop = row["index_loop"]
    peak_values = ATAC_normalised_counts_melted[ATAC_normalised_counts_melted["index"] == index_peak]
    loop_values = loops_counts_melted[loops_counts_melted["index"] == index_loop]
    res = retrieve_p_vals(peak_values, loop_values)
    return pd.concat([row,res])
loops_with_peaks = loops_with_peaks.parallel_apply(annotate_regression, axis = 1)

In [22]:
loops_with_peaks["FDR"] = multitest.multipletests(loops_with_peaks["p_value"], method = "fdr_bh",alpha = 0.1)[1]

In [23]:
loops_with_peaks.to_pickle(f"{base_dir}/integration_analysis/output_files/ATAC_corr_loops.pk")
loops_with_peaks.to_csv(f"{base_dir}/integration_analysis/output_files/ATAC_corr_loops.csv.gz")

In [24]:
loops_with_peaks = pd.read_pickle(f"{base_dir}/integration_analysis/output_files/ATAC_corr_loops.pk")

## now link genes to ATAC-peak and take significant ones 

In [25]:
loops_with_peaks_bed = pbed.BedTool.from_dataframe(loops_with_peaks)

In [26]:
tss_sites = gtf_transcripts[["seqname", "TSS_start","gene_id", "gene_name"]].copy()
tss_sites["TSS_end"] = tss_sites["TSS_start"] + 1
tss_sites = tss_sites.drop_duplicates(["seqname", "TSS_start","gene_id", "gene_name"])
tss_bed = pbed.BedTool.from_dataframe(tss_sites[["seqname", "TSS_start", "TSS_end","gene_id", "gene_name"]])

In [27]:
loops_with_peaks_genes = loops_with_peaks_bed.pairtobed(tss_bed.slop(b = 5000, g = bed_genome_file), type = "either").to_dataframe(header = None, disable_auto_names = True)

In [28]:
loops_with_peaks_genes.columns = loops_with_peaks.columns.to_list() + ["seqname", "TSS_start", "TSS_end","gene_id", "gene_name"]

In [29]:
# remove the genes that were basically overlapping the ATAC side, so keep only long range genes
def drop_genes(row):
    if (row["TSS_end"] > row["START"] - 5000) and (row["TSS_end"] < row["END"] + 5000):
        return False
    if (row["TSS_start"] > row["START"] -5000) and (row["TSS_start"] < row["END"] + 5000):
        return False
    return True
rows_to_keep = loops_with_peaks_genes.apply(drop_genes, axis=1)
loops_with_peaks_genes = loops_with_peaks_genes[rows_to_keep]

In [30]:
significant_loops_with_peaks_genes = loops_with_peaks_genes[loops_with_peaks_genes["FDR"] < 0.1]

In [31]:
significant_loops_with_peaks_genes[significant_loops_with_peaks_genes["gene_name"] == "BCL2L11"]

,chrA,A_start,A_end,chrB,B_start,B_end,index_loop,CHR,START,END,...,R_squared,R_squared_ajd,interaction_mean,peak_mean,FDR,seqname,TSS_start,TSS_end,gene_id,gene_name
88504,chr2,110852500,110855000,chr2,111120000,111122500,14296,chr2,110858286,110858786,...,0.470858,0.465619,20.377721,772.688347,1.389225e-13,chr2,111117670,111127671,ENSG00000153094,BCL2L11
88505,chr2,110852500,110855000,chr2,111120000,111122500,14296,chr2,110858286,110858786,...,0.470858,0.465619,20.377721,772.688347,1.389225e-13,chr2,111118746,111128747,ENSG00000153094,BCL2L11
88506,chr2,110852500,110855000,chr2,111120000,111122500,14296,chr2,110858286,110858786,...,0.470858,0.465619,20.377721,772.688347,1.389225e-13,chr2,111118752,111128753,ENSG00000153094,BCL2L11
88507,chr2,110852500,110855000,chr2,111120000,111122500,14296,chr2,110858286,110858786,...,0.470858,0.465619,20.377721,772.688347,1.389225e-13,chr2,111114378,111124379,ENSG00000153094,BCL2L11
88508,chr2,110852500,110855000,chr2,111120000,111122500,14296,chr2,110858286,110858786,...,0.470858,0.465619,20.377721,772.688347,1.389225e-13,chr2,111115914,111125915,ENSG00000153094,BCL2L11
88509,chr2,110852500,110855000,chr2,111120000,111122500,14296,chr2,110858286,110858786,...,0.470858,0.465619,20.377721,772.688347,1.389225e-13,chr2,111115929,111125930,ENSG00000153094,BCL2L11


In [32]:
loops_with_peaks_genes.to_pickle(f"{base_dir}/integration_analysis/output_files/ATAC_corr_loops_with_genes_ALL.pk")
loops_with_peaks_genes.to_csv(f"{base_dir}/integration_analysis/output_files/ATAC_corr_loops_with_genes_ALL.csv")

significant_loops_with_peaks_genes.to_pickle(f"{base_dir}/integration_analysis/output_files/ATAC_corr_loops_with_genes.pk")
significant_loops_with_peaks_genes.to_csv(f"{base_dir}/integration_analysis/output_files/ATAC_corr_loops_with_genes.csv")

## finally just search the GWAS datasets
this is just first pass, because the ideally we wanted the SNPs that have loopQTL or allelic imbalance, but this one doesn't filter for that, it just overlaps the peaks

In [ ]:
def annotate_loops(file, name):
    snp_bed = pbed.BedTool(file)
    good_peaks = pbed.BedTool.from_dataframe(loops_with_peaks_genes[["CHR","START","END", "index_peak"]].drop_duplicates())
    snp_bed_with_peaks = snp_bed.intersect(good_peaks, wa = True, wb = True).to_dataframe(header = None, disable_auto_names=True)
    snp_bed_with_peaks.columns = "chr start end name score CHR START END index_peak".split()
    snp_bed_with_peaks["loci"] = snp_bed_with_peaks["name"].str.split("_").str[-1]
    snp_bed_with_peaks["snp"] = snp_bed_with_peaks["name"].str.split("_").str[0]
    SNPs_with_loop_with_gene = snp_bed_with_peaks["chr start end loci snp score index_peak".split()].merge(loops_with_peaks_genes, on = "index_peak")
    # filter allelic imbalance
    SNPs_with_loop_with_gene = SNPs_with_loop_with_gene[SNPs_with_loop_with_gene["snp"].isin(all_SNPs_all["ID"])]
    SNPs_with_loop_with_gene.to_csv(f"{base_dir}/integration_analysis/output_files/ATAC_loop_genes_{name}.csv")

annotate_loops(f"{base_dir}/metadata/GWAS_files/tsoi2017_LD_0.8_hg38.bed", "psoriasis_tsoi2017")
annotate_loops(f"{base_dir}/metadata/GWAS_files/RAmetagwas_all_hg38.ld.bed", "RAmeta")
annotate_loops(f"{base_dir}/metadata/GWAS_files/PsA_vs_controls_metagwas_significant.ld.hg38.bed", "PsA_meta")
annotate_loops(f"{base_dir}/metadata/GWAS_files/suggestive_snps_hg38_ld.bed", "JIA_suggestive")
annotate_loops(f"{base_dir}/metadata/GWAS_files/JIA_credible_snps_hg38.bed", "JIA_credible")
annotate_loops(f"{base_dir}/metadata/GWAS_files/elena_hg38.ld.bed", "SSc_elena")



In [86]:
res

,chr,start,end,loci,snp,score,index_peak,chrA,A_start,A_end,...,R_squared,R_squared_ajd,interaction_mean,peak_mean,FDR,seqname,TSS_start,TSS_end,gene_id,gene_name
0,chr1,67333211,67333212,rs4655698,rs11209051,1.000000,3723,chr1,67332500,67335000,...,0.603381,0.599454,24.884601,297.983558,2.057858e-19,chr1,67425415,67435416,ENSG00000142864,SERBP1
1,chr1,67333211,67333212,rs4655698,rs11209051,1.000000,3723,chr1,67332500,67335000,...,0.603381,0.599454,24.884601,297.983558,2.057858e-19,chr1,67425386,67435387,ENSG00000142864,SERBP1
2,chr1,67333211,67333212,rs4655698,rs11209051,1.000000,3723,chr1,67332500,67335000,...,0.603381,0.599454,24.884601,297.983558,2.057858e-19,chr1,67425399,67435400,ENSG00000142864,SERBP1
3,chr1,67333211,67333212,rs4655698,rs11209051,1.000000,3723,chr1,67332500,67335000,...,0.603381,0.599454,24.884601,297.983558,2.057858e-19,chr1,67425372,67435373,ENSG00000142864,SERBP1
4,chr1,198614891,198614892,rs28398409,rs28398409,1.000000,7724,chr1,198160000,198162500,...,0.060059,0.050753,12.508455,138.362658,7.029026e-02,chr1,198151963,198161964,ENSG00000151414,NEK7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,chr4,122578589,122578590,rs6814280,rs13140464,0.882487,76897,chr4,122577500,122580000,...,0.151663,0.143263,10.051215,835.882068,6.222326e-04,chr4,122917442,122927443,ENSG00000170917,NUDT6
81,chr4,122578589,122578590,rs6814280,rs13140464,0.882487,76897,chr4,122577500,122580000,...,0.151663,0.143263,10.051215,835.882068,6.222326e-04,chr4,122918074,122928075,ENSG00000145375,SPATA5
82,chr5,56148855,56148856,rs7731626,rs7731626,1.000000,79654,chr5,55985000,55990000,...,0.765395,0.763073,47.579696,187.216611,5.159165e-30,chr5,55989993,55999994,ENSG00000134352,IL6ST
83,chr5,56148855,56148856,rs7731626,rs7731626,1.000000,79654,chr5,55985000,55990000,...,0.765395,0.763073,47.579696,187.216611,5.159165e-30,chr5,55989964,55999965,ENSG00000134352,IL6ST
